In [ ]:
#Installing necessary libraries
!pip install pandas
!pip install tensorflow
!pip install keras
!pip install -U scikit-learn

In [17]:
#Importing libraries
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from keras.optimizers import RMSprop

In [2]:
#loading train images info from data/train.csv
train_df = pd.read_csv('data/train.csv')
image_names = train_df['ImageId'].values
labels = train_df['ClassId'].values

In [3]:
#loading train images
images = []
for image_name in image_names:
    image_path = f'data/train_images/{image_name}'
    image = cv2.imread(image_path)
    img = cv2.resize(image,(64,64)) #Resize images to (64,64)
    images.append(img) #Add image to "images" list

In [4]:
#Spliting the data into training and validation (80% for train and 20% for test)
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
# Convert the images and labels to numpy arrays
train_images = np.array(train_images)
val_images = np.array(val_images)
train_labels = to_categorical(train_labels - 1)  #Convert labels to one-hot encoding
val_labels = to_categorical(val_labels - 1)  #Convert labels to one-hot encoding

# Desired model

In [18]:
#Specify the height, width and number of image channels
image_height = 64
image_width = 64
num_channels = 3

#Specify the number of labels
num_classes = 4

#CNN model with
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [16]:
#Training the model
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=32, epochs=10, validation_data=(val_images, val_labels))

Epoch 1/10
178/178 [==============================] - 7s 37ms/step - loss: 0.7496 - accuracy: 0.7225 - val_loss: 0.6386 - val_accuracy: 0.7435
Epoch 2/10
178/178 [==============================] - 7s 36ms/step - loss: 0.7200 - accuracy: 0.7274 - val_loss: 0.7104 - val_accuracy: 0.7301
Epoch 3/10
178/178 [==============================] - 7s 36ms/step - loss: 0.7061 - accuracy: 0.7297 - val_loss: 0.6657 - val_accuracy: 0.7470
Epoch 4/10
178/178 [==============================] - 6s 36ms/step - loss: 0.6700 - accuracy: 0.7451 - val_loss: 0.7130 - val_accuracy: 0.7393
Epoch 5/10
178/178 [==============================] - 6s 36ms/step - loss: 0.6864 - accuracy: 0.7354 - val_loss: 0.6400 - val_accuracy: 0.7308
Epoch 6/10
178/178 [==============================] - 6s 36ms/step - loss: 0.6454 - accuracy: 0.7415 - val_loss: 0.6006 - val_accuracy: 0.7541
Epoch 7/10
178/178 [==============================] - 6s 36ms/step - loss: 0.6678 - accuracy: 0.7414 - val_loss: 0.6857 - val_accuracy: 0.7449

# using VGG16

In [19]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

In [20]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_height, image_width, num_channels))

58889256/58889256 [==============================] - 110s 2us/step


In [33]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [34]:
for layer in base_model.layers:
    layer.trainable = False

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=32, epochs=10, validation_data=(val_images, val_labels))

Epoch 1/10
178/178 [==============================] - 26s 137ms/step - loss: 0.3896 - accuracy: 0.8460 - val_loss: 0.8662 - val_accuracy: 0.8013
Epoch 2/10
178/178 [==============================] - 24s 134ms/step - loss: 0.3245 - accuracy: 0.8813 - val_loss: 0.9594 - val_accuracy: 0.8118
Epoch 3/10
178/178 [==============================] - 24s 135ms/step - loss: 0.2816 - accuracy: 0.8876 - val_loss: 0.8731 - val_accuracy: 0.8111
Epoch 4/10
178/178 [==============================] - 24s 135ms/step - loss: 0.2584 - accuracy: 0.8915 - val_loss: 0.8170 - val_accuracy: 0.8111
Epoch 5/10
178/178 [==============================] - 24s 135ms/step - loss: 0.2471 - accuracy: 0.8945 - val_loss: 0.7987 - val_accuracy: 0.8097
Epoch 6/10
178/178 [==============================] - 24s 135ms/step - loss: 0.2457 - accuracy: 0.8973 - val_loss: 0.8390 - val_accuracy: 0.8147
Epoch 7/10
178/178 [==============================] - 24s 135ms/step - loss: 0.2301 - accuracy: 0.8964 - val_loss: 0.7724 - val_ac

# using ResNet50

In [24]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

In [25]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_height, image_width, num_channels))

94765736/94765736 [==============================] - 130s 1us/step


In [36]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [37]:
for layer in base_model.layers:
    layer.trainable = False

In [38]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=32, epochs=10, validation_data=(val_images, val_labels))

Epoch 1/10
178/178 [==============================] - 27s 141ms/step - loss: 0.4004 - accuracy: 0.8552 - val_loss: 0.7669 - val_accuracy: 0.8013
Epoch 2/10
178/178 [==============================] - 24s 136ms/step - loss: 0.3024 - accuracy: 0.8797 - val_loss: 1.1864 - val_accuracy: 0.8034
Epoch 3/10
178/178 [==============================] - 24s 137ms/step - loss: 0.3277 - accuracy: 0.8813 - val_loss: 0.8747 - val_accuracy: 0.8090
Epoch 4/10
178/178 [==============================] - 24s 137ms/step - loss: 0.2686 - accuracy: 0.8906 - val_loss: 0.6563 - val_accuracy: 0.8076
Epoch 5/10
178/178 [==============================] - 24s 137ms/step - loss: 0.2565 - accuracy: 0.8945 - val_loss: 1.3891 - val_accuracy: 0.8147
Epoch 6/10
178/178 [==============================] - 24s 137ms/step - loss: 0.3291 - accuracy: 0.8927 - val_loss: 0.8797 - val_accuracy: 0.8048
Epoch 7/10
178/178 [==============================] - 24s 136ms/step - loss: 0.2561 - accuracy: 0.8989 - val_loss: 0.8773 - val_ac

# testing

In [ ]:
#Loading the test images
test_df = pd.read_csv('data/test.csv')
test_image_names = test_df['ImageId'].values

In [ ]:
#loading test images
test_images = []
for image_name in test_image_names:
    image_path = f'data/test_images/{image_name}'
    image = cv2.imread(image_path)
    #Add image to "test_images" list
    test_images.append(image)

In [ ]:
#Convert the test images to numpy array
test_images = np.array(test_images)

#Predict the test set labels
test_predictions = model.predict(test_images)

In [ ]:
test_predictions_labels = (np.argmax(test_predictions, axis=1)) + 1

In [ ]:
test_predictions_labels

In [ ]:
#Saving the model
model.save('model.h5')

In [ ]:
#Loading the saved model
model = load_model('model.h5')